<a href="https://colab.research.google.com/github/Lawler-Research-Group/Lawler-Research-Group.github.io/blob/master/BI_TACS_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymc
import pymc as pm
import pandas as pd
import statsmodels.api as sm
%matplotlib inline
import math
from IPython.core.pylabtools import figsize
import os
import re
import matplotlib.pyplot as plt
import arviz as az
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")
import numpy as np
import random
from math import sqrt
from scipy.optimize import curve_fit
from numba import njit
from matplotlib.pyplot import figure
#import aesara.tensor as at
#from aesara import function
# use the three commands below only if you're using google colab
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


In [3]:
def extract_num(string):
  result = re.findall(r"[-+]?\d*\.\d+|\d+", string)
  floats = []
  for i in range(len(result)):
    floats.append(float(result[i]))
  if len(floats) == 1:
    return floats[0]
  else:    
    return floats

def cs_encoding(x):
  x.replace('"', '')
  x.replace('=','')
  x.replace('>','')
  x = ''.join(filter(str.isalnum, x))
  lst = []
  for i in range(int(len(x)/2)):
    if x[2*i]=='Z' and x[2*i+1]=='1':
      lst.append(int(0))
    elif x[2*i]=='Z' and x[2*i+1]=='0':
      lst.append(int(1))
    elif x[2*i]=='X' and x[2*i+1]=='1':
      lst.append(int(2))
    elif x[2*i]=='X' and x[2*i+1]=='0':
      lst.append(int(3))
    elif x[2*i]=='Y' and x[2*i+1]=='1':
      lst.append(int(4))
    elif x[2*i]=='Y' and x[2*i+1]=='0':
      lst.append(int(5))
  return lst 

def data_load(prefix='/content/drive/MyDrive/Datasets/187.zip/', lops=False):
  files = [f for f in os.listdir(prefix) if (('ti' in f) == False and ('mag' in f) == False)]
  df_lst = []
  c = 0
  for f in range(len(files)):
    hx_time = extract_num(files[f])
    if lops==True and hx_time > 1 and c <= 40:
      c += 1
      continue
    df = pd.read_csv(prefix+'/'+files[f], '\t', names=['spin_measurements'])
    df['spin_measurements'] = df['spin_measurements'].apply(lambda x: cs_encoding(x)) 
    df['hx'] = hx_time
    df_lst.append(df)
  return df_lst        

def process_cs_data(data_frame):
  state_list = []
  h = len(data_frame)
  T = len(data_frame[0])
  N = len(data_frame[0]['spin_measurements'].loc[0])
  states = np.zeros((h ,T, N))
  hx_values = []
  # times = []
  for i in range(len(data_frame)):
    state_list.append(data_frame[i]['spin_measurements'].apply(lambda x: np.array(x)).to_numpy())
    hx_values.append(data_frame[i]['hx'].loc[0])
    # times.append(data_frame[i][j]['time'].loc[0])
  for i in range(len(state_list)):
   for j in range(len(state_list[0])):
     states[i][j][0:N] = state_list[i][j][0:N]
  idx = np.argsort(hx_values)
  states = states[idx]
  hx_values = np.array(hx_values)[idx] 
  # times = np.array(times)[idx]
  return states.astype("int"), hx_values     

In [4]:
# pauli matrix eigenvectors
z_up = np.array([1,0])
z_down = np.array([0,1])
x_up = (1/sqrt(2))*np.array([1,1])
x_down = (1/sqrt(2))*np.array([1,-1])
y_up = (1/sqrt(2))*np.array([1, 1j])
y_down = (1/sqrt(2))*np.array([1,-1j]) 
cs_eigvecs = [z_up,z_down,x_up,x_down,y_up,y_down]      # the order here is important
z_eigvecs = [z_down, z_up]

def sigma(vec):
  return 3*np.outer(vec, np.conjugate(vec)) - np.identity(len(vec))

def trace_sigma(vec1,vec2):
  return np.trace(np.matmul(sigma(vec1), sigma(vec2)))

tr_mat_cs = np.zeros((len(cs_eigvecs),len(cs_eigvecs)))
for i in range(len(cs_eigvecs)):
  for j in range(len(cs_eigvecs)):
    tr_mat_cs[i,j] = trace_sigma(cs_eigvecs[i], cs_eigvecs[j])
# this 6*6 matrix is the trace matrix that gives. so tr_mat_cs[2,3] would give trace(sigma(x_up),sigma(x_down))    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: ComplexWarning: Casting complex values to real discards the imaginary part


In [5]:
@njit
def ren_x(rho, n, trace_matrix=tr_mat_cs):
  # here rho is 2D np array which represents a TACS, each row is a snapshot. Trace_matrix is the 6*6 matrix which holds the trace(sigma1*sigma2) values.
  # n is the reduced no. of qubits 
  ren = 0
  c = 1
  N, L = np.shape(rho)
  if n%2 == 0:
    left = int(10 - n/2)
    right = int(10 + n/2)
  else:
    if n == 1:
      left = 10
      right = 11
    else:  
      left = int(10 - (n/2 - 0.5))
      right = int(10 + (n/2 + 0.5))      
  for i in range(N):
    for j in range(N):
      for g in range(left,right):
          c = c * trace_matrix[rho[i,g],rho[j,g]] 
      ren = ren + c  
      c = 1  
  return ren / (N**2) 

@njit
def ren_all(rho_Ts, n, x=1):
  ren_x(rho_Ts[0],n=1)
  ren_S = np.zeros(len(rho_Ts))
  for j in range(len(rho_Ts)):
    ren_S[j] = ren_x(rho_Ts[j],n=n)
  return ren_S

In [7]:
def get_tacs(hx, no_shots, states, hx_values):
  i = list(hx_values).index(hx)
  state = states[i]
  inds = np.random.randint(len(state)-1, size=no_shots)
  tacs = state[inds]
  return tacs

def data_gen(hx, no_points, states, hx_values, n_vals=np.arange(1,6), N_vals=np.arange(100,500)):
  gamma_vals = np.zeros((no_points,3))
  for i in range(no_points):
    n = n_vals[np.random.randint(len(n_vals))]
    N_shots = N_vals[np.random.randint(len(N_vals))]
    CS = get_tacs(hx, N_shots,states=states,hx_values=hx_values)
    gam = ren_x(CS,n=n)
    gamma_vals[i] = n, N_shots, gam
  return gamma_vals 

def data_gen2(hx, states, hx_values, no_n=5, no_N=500, n_low=1, n_high=5, N_low=10000, N_high=20000):
  n_vals = np.linspace(n_low,n_high,no_n)
  N_vals = np.linspace(N_low,N_high,no_N)
  gamma_vals = np.zeros((len(n_vals),len(N_vals)))
  for i in range(no_n):
    for j in range(no_N):
      n, N = round(n_vals[i]), round(N_vals[j])
      CS = get_tacs(hx, N, states=states, hx_values=hx_values)
      gam = ren_x(CS,n=n)
      gamma_vals[i,j] = gam
      n_vals[i], N_vals[j] = n, N
  return n_vals, N_vals, gamma_vals   

In [8]:
df = data_load()
states, hx_values = process_cs_data(df)
np.shape(states)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  """Entry point for launching an IPython kernel.


(187, 500, 20)

In [ ]:
# DO NOT TOUCH THIS CELL UNLESS
a_hold = []
b_hold = []
hx_hold = []
hxs = hx_values[0:2]
#hxs = hxs[np.random.randint(len(hxs)-1, size=40)]
for h in range(len(hxs)):
  dat = data_gen(hxs[h], 300, n_vals=np.arange(1,6))
  n_vals = dat[:,0]
  N_vals = dat[:,1]
  gam_vals = dat[:,2]
  with pm.Model() as model:
    a = pm.HalfNormal("a",sigma=5.0)
    b = pm.HalfNormal("b",sigma=5.0)
    c = pm.HalfNormal("c", sigma=5.0)
    d = pm.HalfNormal("d",sigma=5.0)
    e = pm.HalfNormal("e", sigma=50.0)
    f = pm.HalfNormal("f", sigma=5.0)
    #avg = a*pm.math.exp(-n_vals/b)
    avg = a*pm.math.exp(-n_vals/b) + c*pm.math.exp(n_vals/d) * (1/N_vals)
    var = e*pm.math.exp(n_vals/f) / (N_vals**2)
    likelihood = pm.Normal("likelihood",mu=avg, sigma=pm.math.sqrt(var), observed=gam_vals)
    trace = pm.sample(step=pm.NUTS(),draws=20000)
  df = az.summary(trace)  
  a_mu, a_sig = df.loc['a']['mean'], df.loc['a']['sd']  
  b_mu, b_sig = df.loc['b']['mean'], df.loc['b']['sd'] 
  a_hold.append(np.array([a_mu,a_sig]))
  b_hold.append(np.array([b_mu,b_sig]))
  hx_hold.append(hxs[h])

In [1]:
#If you want to see the MC results 
#axes = az.plot_trace(trace)
#fig = axes.ravel()[0].figure

In [ ]:
a_hold = np.array(a_hold)
b_hold = np.array(b_hold)
hx_hold = np.array(hx_hold)

In [ ]:
plt.errorbar(hx_hold, 1/ b_hold[:,0], yerr=b_hold[:,1]/(b_hold[:,0]**2), marker='o', ls='none')
#plt.scatter(a_hold[:,1],b_hold[:,1])
plt.xlabel(r"$h_x$")
plt.ylabel(r'$\frac{1}{b}$',fontsize=20)
#plt.legend(loc='best')
#plt.savefig('/content/drive/MyDrive/Datasets/Plots/renyi/new_plot_olddatalow.pdf',format='pdf',bbox_inches='tight')
plt.show()